# Bibliotecas 

In [1]:
import sys
sys.path.append('/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/')


import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

import hepherolib.analysis as ana
from lgbm.controllers_lgb import LGBLearner, LGBModel

ana.start()
plt.style.use("default")

# Configuração

In [2]:
period = 'APV_16'
year_style = 2016
dataset_year = "APV_2016"
basedir = '/eos/user/t/thenriqu/Dark_Matter/Amostras/hhdmAnalysis_deepJet_Regions/datasets'
dataset_name = basedir.split('/')[-2]

# Data folder
data_path = f"./data/{dataset_name}/{dataset_year}"

# Setup output folders
models_path = f"./models/{dataset_name}/{dataset_year}"
Path(models_path).mkdir(parents=True, exist_ok=True)

# Lendo os dados

In [3]:
model_name = "LGB_multi_signal"

X_train = pd.read_csv(f"{data_path}/{model_name}-train-data.csv")
X_test = pd.read_csv(f"{data_path}/{model_name}-test-data.csv")

W_train = X_train.pop("modelWeight").values
W_test = X_test.pop("modelWeight").values

Y_train = X_train.pop("Label").values
Y_test = X_test.pop("Label").values

evtWeight_train = X_train.pop("evtWeight").values
evtWeight_test = X_test.pop("evtWeight").values

In [4]:
X_train.head()

,LeadingLep_pt,LepLep_deltaM,LepLep_deltaR,LepLep_pt,MET_LepLep_Mt,MET_LepLep_deltaPhi,MET_pt,MT2LL,Nbjets,TrailingLep_pt
0,127.877914,5.665382,1.004920,184.44044,197.350770,2.764841,54.709740,96.820950,1,77.357370
1,113.009460,3.599693,1.009094,178.61588,147.659620,1.591687,59.785347,63.220463,1,79.000790
2,97.949240,22.735489,2.811170,67.28066,119.337814,3.041180,53.051918,14.674115,1,33.556896
3,311.434630,4.510574,0.451174,423.52713,882.859400,3.096336,460.324680,393.019100,1,120.251110
4,144.094960,24.149925,2.172507,130.71048,313.220800,2.877176,190.960710,61.069447,2,28.793385


In [5]:
W_train

array([1.24207517e+00, 7.10710006e-04, 1.56460205e+00, ...,
       2.76548358e-01, 1.13629105e-03, 1.06929979e-01])

In [6]:
Y_train

array([0, 0, 0, ..., 0, 0, 0])

# Train

In [7]:
path1 = '/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/Analysis_Regions/Analysis_2018/models/hhdmAnalysis_deepJet_Regions/2018/'


In [8]:
with open(path1 + 'LGB_multi_signal-bst-params.json', 'r') as f:
    best_params = json.load(f)

print(best_params)

{'subsample': 0.9, 'n_estimators': 9000, 'min_child_weight': 6, 'min_child_samples': 100, 'max_depth': 9, 'learning_rate': 0.008, 'eval_metric': 'error', 'colsample_bytree': 0.8}


In [10]:
# params = { 'subsample': 0.9,
#           'n_estimators': 9000,
#           'min_child_weight': 6,
#           'min_child_samples': 100,
#           'max_depth': 9,
#           'learning_rate': 0.008,
#           'eval_metric': 'error',
#           'colsample_bytree': 0.8
# }

# Pode se ler através do arquivos .json salvo no código 2_find_hyperparams_lgb.ipynb
params = json.load(open(f"{models_path}/{model_name}-bst-params.json"))

num_boost_round = params.pop('n_estimators')
features = ["LeadingLep_pt", "LepLep_pt", "LepLep_deltaR", "LepLep_deltaM", "MET_pt", "MET_LepLep_Mt", "MET_LepLep_deltaPhi", "TrailingLep_pt", "MT2LL", "Nbjets"]

learner = LGBLearner(X_train, Y_train, W_train, features)
learner.train(params, num_boost_round)
learner.save_model(f"{models_path}/{model_name}-clf.model")
# learner.dump_model(f"{models_path}/{model_name}-raw.model")

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002891 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2302
[LightGBM] [Info] Number of data points in the train set: 716588, number of used features: 10
[LightGBM] [Info] Start training from score 0.666667


# Nofiticar quando o Notebook terminar

In [ ]:
from IPython.display import clear_output, display, HTML, Javascript

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Processo Completo!";
  window.speechSynthesis.speak(msg);
  alert("Processo Completo!")
"""))